In [1]:
import pandas as pd

df = pd.read_csv('dataset_cleaned.csv')
print(df.head())
print(df.info())
print(df.describe(include='all'))
print(df.columns)


  transaction_id user_id        date transaction_type       category   amount  \
0          T4999    U018  2023-04-25          Expense      Education   3888.0   
1         T12828    U133  2022-08-05          Expense           Rent    649.0   
2          T7403    U091  2023-12-31           Income      Freelance  13239.0   
3          T7495    U088  2022-10-28          Expense  Entertainment   2287.0   
4         T12465    U042  2024-04-11          Expense           Food   4168.0   

  payment_mode   location          notes  
0         Card  Ahmedabad  Movie tickets  
1      Unknown  Hyderabad         asdfgh  
2         Cash        BAN          Books  
3         Card  Hyderabad       No notes  
4      Unknown    Unknown           test  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11875 entries, 0 to 11874
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    11875 non-null  object 
 1   u

In [3]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.day_name()


print(df[['date', 'year', 'month', 'day', 'weekday']].head(10))


        date  year  month  day    weekday
0 2023-04-25  2023      4   25    Tuesday
1 2022-08-05  2022      8    5     Friday
2 2023-12-31  2023     12   31     Sunday
3 2022-10-28  2022     10   28     Friday
4 2024-04-11  2024      4   11   Thursday
5 2024-12-12  2024     12   12   Thursday
6 2023-11-10  2023     11   10     Friday
7 2022-04-15  2022      4   15     Friday
8 2022-06-29  2022      6   29  Wednesday
9 2022-02-24  2022      2   24   Thursday


In [6]:
category_mapping = {
    'food': 'food',
    'grocery': 'food',
    'rent': 'rent',
    'housing': 'rent',
    'travel': 'travel',
    'transport': 'travel',
    'hotel': 'travel',
    'flight': 'travel',
    'shopping': 'shopping',
    'clothing': 'shopping',
    'apparel': 'shopping',
    'entertainment': 'entertainment',
    'education': 'education',
    'salary': 'income',
    'freelance': 'income',
    'bonus': 'income',
    'utilities': 'bills',
    'electricity': 'bills',
    'investment': 'investment',
    'savings': 'investment',
    'others': 'other',
    'unknown': 'other'
}

def improved_map_category(cat):
    cat = str(cat).lower()
    return category_mapping.get(cat, 'other')

df['category_group'] = df['category'].apply(improved_map_category)

print(df[['category', 'category_group']].drop_duplicates().sort_values('category_group'))


         category category_group
7       Utilities          bills
0       Education      education
3   Entertainment  entertainment
4            Food           food
2       Freelance         income
5          Salary         income
59          Bonus         income
22     Investment     investment
43        Savings     investment
6          Others          other
26         Health          other
1            Rent           rent
20         Travel         travel


In [14]:
df = pd.read_csv('dataset_cleaned.csv')
print("Duplicates before removal:", df.duplicated().sum())


Duplicates before removal: 686


In [ ]:
df = pd.read_csv('dataset_cleaned.csv')

df['date'] = pd.to_datetime(df['date'], errors='coerce')

df['completeness'] = (
    df['amount'].notnull().astype(int) +
    df['notes'].apply(lambda x: 0 if pd.isna(x) or str(x).strip() == '' else 1) +
    df['payment_mode'].apply(lambda x: 0 if x == 'Unknown' else 1) +
    df['location'].apply(lambda x: 0 if x == 'Unknown' else 1)
)

df.sort_values(by=['transaction_id', 'completeness', 'date'], ascending=[True, False, False], inplace=True)

df = df.drop_duplicates(subset='transaction_id', keep='first')

df.drop(columns=['completeness'], inplace=True)

print(df['transaction_id'].duplicated().sum())



0


In [16]:
category_mapping = {
    'food': 'food',
    'grocery': 'food',
    'rent': 'rent',
    'housing': 'rent',
    'travel': 'travel',
    'transport': 'travel',
    'hotel': 'travel',
    'flight': 'travel',
    'shopping': 'shopping',
    'clothing': 'shopping',
    'apparel': 'shopping',
    'entertainment': 'entertainment',
    'education': 'education',
    'salary': 'income',
    'freelance': 'income',
    'bonus': 'income',
    'utilities': 'bills',
    'electricity': 'bills',
    'investment': 'investment',
    'savings': 'investment',
    'others': 'other',
    'unknown': 'other'
}

def improved_map_category(cat):
    cat = str(cat).lower()
    return category_mapping.get(cat, 'other')

df['category_group'] = df['category'].apply(improved_map_category)


In [ ]:
import numpy as np

df['is_outlier'] = False

group_cols = ['transaction_type', 'category_group']

for group_keys, group_df in df.groupby(group_cols):
    q1 = group_df['amount'].quantile(0.25)
    q3 = group_df['amount'].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outlier_mask = (group_df['amount'] < lower_bound) | (group_df['amount'] > upper_bound)
    df.loc[group_df[outlier_mask].index, 'is_outlier'] = True

print(df[df['is_outlier']].head())

print("Total outliers detected:", df['is_outlier'].sum())



     transaction_id user_id       date transaction_type   category    amount  \
3052          T0021    U081 2023-11-14           Income     Salary  945687.0   
7620          T0123    U085 2021-07-15           Income      Bonus  177959.0   
558           T0163    U098 2021-12-20           Income     Others  749253.0   
1887          T0212    U003 2022-12-10           Income      Bonus  436526.0   
4958          T0290    U037 2021-07-21           Income  Freelance  506415.0   

       payment_mode   location                 notes category_group  \
3052           Card  hyderabad              No notes         income   
7620           Cash  Hyderabad  Monthly rent payment         income   
558   Bank Transfer  AHMEDABAD      Electricity bill          other   
1887           Cash    KOLKATA  Monthly rent payment         income   
4958           Cash    LUCKNOW         Online course         income   

      is_outlier  
3052        True  
7620        True  
558         True  
1887        True

In [ ]:
import numpy as np

for group_keys, group_df in df.groupby(['transaction_type', 'category_group']):
    q1 = group_df['amount'].quantile(0.25)
    q3 = group_df['amount'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    idx = group_df.index
    df.loc[idx, 'amount'] = np.clip(df.loc[idx, 'amount'], lower, upper)


In [ ]:
df['is_outlier'] = False

for group_keys, group_df in df.groupby(['transaction_type', 'category_group']):
    q1 = group_df['amount'].quantile(0.25)
    q3 = group_df['amount'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    outlier_mask = (group_df['amount'] < lower) | (group_df['amount'] > upper)
    df.loc[group_df[outlier_mask].index, 'is_outlier'] = True

print("Remaining outliers after capping:", df['is_outlier'].sum())


Remaining outliers after capping: 0


In [22]:
print(df.head())  # See first 5 rows
print(df.tail())  # See last 5 rows
print(df.info())  # Data types and non-null counts
print(df.describe())  # Numeric stats, especially for 'amount'
print(df.columns)
print("Final row count:", len(df))


     transaction_id user_id       date transaction_type   category   amount  \
9945          T0002    U072 2021-01-14          Expense    Savings   5614.0   
3185          T0003    U097 2022-08-24          Expense     Health    850.0   
7651          T0004    U091 2022-07-01          Expense     Travel   4110.0   
4421          T0006    U036 2021-07-06           Income  Freelance  18326.0   
3098          T0008    U042 2023-12-23          Expense       Food   2569.0   

       payment_mode   location              notes category_group  is_outlier  
9945            UPI  ahmedabad  Restaurant dinner     investment       False  
3185           Card  ahmedabad               misc          other       False  
7651           Card        MUM     Gym membership         travel       False  
4421  Bank Transfer        JAI   Grocery shopping         income       False  
3098           Cash      Delhi      Fixed deposit           food       False  
     transaction_id user_id       date transaction_

In [24]:
print(sorted(df['category'].unique()))
print(df['category'].value_counts())



['Bonus', 'Education', 'Entertainment', 'Food', 'Freelance', 'Health', 'Investment', 'Others', 'Rent', 'Salary', 'Savings', 'Travel', 'Utilities']
category
Food             2249
Rent             1806
Travel           1317
Utilities        1063
Entertainment     893
Education         691
Others            682
Health            416
Investment        326
Freelance         318
Salary            316
Bonus             296
Savings           255
Name: count, dtype: int64
